In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Carga de Datos

In [3]:
postulantesEducacion = pd.read_csv('resources/tar/fiuba_1_postulantes_educacion.csv')
postulantesGenero = pd.read_csv('resources/tar/fiuba_2_postulantes_genero_y_edad.csv')
vistas = pd.read_csv('resources/tar/fiuba_3_vistas.csv')
postulaciones = pd.read_csv('resources/tar/fiuba_4_postulaciones.csv')
avisosOnline = pd.read_csv('resources/tar/fiuba_5_avisos_online.csv')
avisosDetalle = pd.read_csv('resources/tar/fiuba_6_avisos_detalle.csv')

#cargamos zips
avisosDetalleZIP = pd.read_csv('resources/zip/fiuba_6_avisos_detalle.csv')
avisosDetalleMissing = pd.read_csv('resources/zip/fiuba_6_avisos_missing.csv')

postulantesEducacionZip = pd.read_csv('resources/zip/fiuba_1_postulantes_educacion.csv')
postulantesGeneroZip = pd.read_csv('resources/zip/fiuba_2_postulantes_genero_y_edad.csv')

vistasZIP = pd.read_csv('resources/zip/fiuba_3_vistas.csv')

In [3]:
#Set de test
testingSet = pd.read_csv('resources/test_final_100k.csv')

In [ ]:
#Levanto avisos no postulados
avisosNoPostulado = pd.DataFrame();
for i in range(0,15):
    noPostulado = pd.read_csv('nopostulados/nopostulados-'+str(i)+'.csv')
    avisosNoPostulado = pd.concat([noPostulado,avisosNoPostulado]);

avisosNoPostulado.drop('Unnamed: 0',axis=1,inplace=True)
avisosNoPostulado = avisosNoPostulado.drop_duplicates()

# Preparacion de datos

In [272]:
#Mergeo los detalles
detalleTodas = pd.concat([avisosDetalleZIP,avisosDetalleMissing])

In [273]:
#Convierto en date
postulaciones['datePostulacion'] = pd.DatetimeIndex(postulaciones['fechapostulacion']).date

In [274]:
#Mergeo avisos con postulaciones
postulacionesMerged = pd.merge(postulaciones,avisosDetalle,on='idaviso',how='inner');

### Transformo las empresas a numeros

In [ ]:
#Convertimos los nombres de las empresas a numericals

In [275]:
postulacionesMerged['cantidadEmpresa'] = 1;
postulacionesMerged['denominacion_empresa'].fillna('S/E', inplace=True);
cantAvisosPorEmpresa = postulacionesMerged.groupby('denominacion_empresa').sum()
cantAvisosPorEmpresa = cantAvisosPorEmpresa.reset_index()
cantAvisosPorEmpresa = cantAvisosPorEmpresa[['denominacion_empresa','cantidadEmpresa']]
postulacionesMerged.drop('cantidadEmpresa',axis=1,inplace=True)

In [54]:
#Mergeo con las postulaciones

In [276]:
postulacionesConEmpresas = pd.merge(postulacionesMerged,cantAvisosPorEmpresa,how='inner',on='denominacion_empresa')

In [ ]:
#Contamos la cantidad de avisos a los que se postulo

In [277]:
avisosPostulados = postulaciones[['idaviso','idpostulante']]
avisosPostulados['cantPostulaciones'] = 1;
avisosPostuladosGP = avisosPostulados.groupby(['idaviso','idpostulante']).sum()
avisosPostuladosGP = avisosPostuladosGP.reset_index()

C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# Convierto zona, tipo de trabajo y nivel laboral en numerical

In [278]:
avisosPostuladosGP['sepostulo'] = 1

In [279]:
 #Me quedo con pocos porque tengo pocos no vistos
avisosPostuladosSample = avisosPostuladosGP.sample(frac=0.1)

In [280]:
#Concateno los no postulados, con los postulados
datosMergeados = pd.concat([avisosNoPostulado,avisosPostuladosSample])

In [281]:
datosMergeados = datosMergeados.fillna(0)

In [282]:
#Marco los sets de training con el de test
testingSet['testSet'] =1;
datosMergeados['testSet'] = 0;

In [283]:
#Ahora unimos los detalles con los avisos
datosMergeadosConDetalle = pd.merge(datosMergeados,avisosDetalle,how='inner',on='idaviso');
testingSetConDetalle = pd.merge(testingSet,detalleTodas,how='left',on='idaviso');

In [ ]:
#Preparo el set de datos para unirlo y pasarlo a categorical

In [284]:

datosConTestSet = pd.concat([datosMergeadosConDetalle,testingSetConDetalle]);
#datosConTestSetFiltrado = datosConTestSet.drop(['titulo','descripcion','ciudad','mapacalle','nombre_area'],axis=1)
datosConTestSetFiltrado = datosConTestSet;

In [285]:
datosOHC = pd.concat([datosConTestSetFiltrado,pd.get_dummies(datosConTestSetFiltrado['nombre_zona'], prefix='zona')],axis=1)
datosOHC = pd.concat([datosOHC,pd.get_dummies(datosOHC['tipo_de_trabajo'], prefix='tipo')],axis=1)
datosOHC = pd.concat([datosOHC,pd.get_dummies(datosOHC['nivel_laboral'], prefix='nivel')],axis=1)

datosOHC.drop(['nivel_laboral','tipo_de_trabajo','nombre_zona'],axis=1,inplace=True)

In [286]:
datosOHC['denominacion_empresa'].fillna('S/E', inplace=True)
datosOHC = pd.merge(datosOHC,cantAvisosPorEmpresa,how='left',on='denominacion_empresa')
datosOHC.drop('denominacion_empresa',axis=1,inplace=True);

### Preparo Genero y Estudios

In [287]:
educacionTodas = pd.concat([postulantesEducacion,postulantesEducacionZip])
generoTodas = pd.concat([postulantesGenero,postulantesGeneroZip])

In [288]:
import datetime as dt

generoTodas['fechanacimiento'] = pd.to_datetime(generoTodas['fechanacimiento'], format='%Y-%m-%d', errors='coerce')
generoTodas = generoTodas.dropna(subset=['fechanacimiento'])

hoy = pd.Timestamp(dt.datetime.now())
generoTodas['edad'] = (hoy - generoTodas['fechanacimiento']).astype('<m8[Y]')

#generoTodasFiltered = generoTodas.loc[((generoTodas['edad'] >= 17) & (generoTodas['edad'] < 80 )), :]


In [ ]:
#Convierto a numerical el sexo

In [289]:

postulantesGeneroOHC = pd.concat([generoTodas,pd.get_dummies(generoTodas['sexo'], prefix='sexo')],axis=1)
postulantesGeneroOHC = postulantesGeneroOHC[postulantesGeneroOHC['sexo_0.0'] == 0]
postulantesGeneroOHC = postulantesGeneroOHC[['idpostulante','sexo_FEM','sexo_MASC','sexo_NO_DECLARA','edad']]

postulantesGeneroOHC.drop_duplicates(subset='idpostulante',inplace=True)

In [ ]:
##Trabajo con la educacion

In [290]:
postulantesEducacion['nombreEstado'] = postulantesEducacion['nombre'] +" - " + postulantesEducacion['estado']
educacionOHC = pd.concat([postulantesEducacion,pd.get_dummies(postulantesEducacion['nombreEstado'], prefix='educacion')],axis=1)
educacionOHC.drop(['nombre','estado','nombreEstado'],axis=1,inplace=True)

In [291]:
educacionOHCGP = educacionOHC.groupby('idpostulante').sum()
educacionOHCGP = educacionOHCGP.reset_index()

In [292]:
generoYEducacionOHC  = pd.merge(postulantesGeneroOHC,educacionOHCGP,how='inner',on='idpostulante')

### Cuento cantidad de vistas de un aviso

In [293]:
#Set de testing
#vistasZIP['timestamp'] = pd.to_datetime(vistasZIP['timestamp'])
vistasZIP['dateVista'] = pd.DatetimeIndex(vistasZIP['timestamp']).date

In [294]:
vistasZIP['cantidadVistos'] = 1;
vistasZIP.drop('timestamp',axis=1,inplace=True)
#vistasZIPGP = vistasZIP.groupby(['idAviso','idpostulante']).sum()
vistasZIPGP = vistasZIP.groupby(['idAviso']).sum()

In [ ]:
#Set de training

In [295]:
vistas['cantidadVistos'] =1;
#vistasGP = vistas.groupby(['idAviso','idpostulante']).sum()
vistasGP = vistas.groupby(['idAviso']).sum()

In [296]:
vistasGP = vistasGP.reset_index()
vistasGP['idaviso']  = vistasGP['idAviso']
vistasGP.drop('idAviso',axis=1,inplace=True);

vistasZIPGP = vistasZIPGP.reset_index()
vistasZIPGP['idaviso']  = vistasZIPGP['idAviso']
vistasZIPGP.drop('idAviso',axis=1,inplace=True);

###  Nombre Area

In [304]:
#Unimos los detalles de los avisos con los avisos del set de test
detallesTodosPosta = pd.concat([avisosDetalle, detalleTodas])

In [305]:
detTodas = detallesTodosPosta[['nombre_area']];

In [306]:
detTodas['cantidadNombre'] = 1
detTodas['cantidadNombre'].fillna('Otros')
detalleTodasGP = detTodas.groupby('nombre_area').sum()

C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [308]:
detalleTodasGP = detalleTodasGP.reset_index()
nombreConCantidad = detalleTodasGP[['nombre_area','cantidadNombre']]

### Mergeamos Vistas y Nombre Area

In [309]:
datosForTesting = datosOHC[datosOHC['testSet'] == 1]
datosForTraining =datosOHC[datosOHC['testSet'] == 0]

In [310]:
datosForTesting.columns.size

44

In [311]:
datosForTraining.columns.size

44

In [ ]:
#Uno con genero y educacion

In [312]:
datosForTrainingConGyE = pd.merge(datosForTraining,generoYEducacionOHC,on='idpostulante',how='inner')
datosForTestingConGyE = pd.merge(datosForTesting,generoYEducacionOHC,on='idpostulante',how='left')

In [ ]:
#Mergeamos con los nombres

In [313]:
datosForTrainingConNombre = pd.merge(datosForTrainingConGyE,nombreConCantidad,on='nombre_area',how='inner')
datosForTestingConNombre = pd.merge(datosForTestingConGyE,nombreConCantidad,on='nombre_area',how='left')

In [314]:
datosForTrainingConNombre['cantidadNombre'] = datosForTrainingConNombre['cantidadNombre'].fillna(0)
datosForTestingConNombre['cantidadNombre'] = datosForTestingConNombre['cantidadNombre'].fillna(0)

In [ ]:
#Mergeamos vistas

In [315]:
datosConVistas = pd.merge(datosForTrainingConNombre, vistasGP,on=['idaviso'],how='inner')
testingConVistas = pd.merge(datosForTestingConNombre,vistasZIPGP,on=['idaviso'],how='left')

In [316]:
testingConVistas['cantidadVistos'] = testingConVistas['cantidadVistos'].fillna(0)
datosConVistas['cantidadVistos'] = datosConVistas['cantidadVistos'].fillna(0)

# Machine Learning

In [244]:
import pandas
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn import svm
from sklearn import tree
from sklearn import gaussian_process
from sklearn import neural_network

#Pre-procesamiento
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline

In [232]:
datosML = datosConVistas
testingML = testingConVistas

In [317]:
datosSinPostulacion = datosML.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet','nombre_area','descripcion','titulo','mapacalle','ciudad'],axis=1)
datosToPredict = testingML.drop(['idpais','sepostulo','idaviso','idpostulante','id','cantPostulaciones','testSet','nombre_area','descripcion','titulo','mapacalle','ciudad'],axis=1)

In [318]:
datosSinPostulacion.columns.size

59

In [319]:
datosToPredict.columns.size

59

In [320]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
datosForTrainingSinPostuSinNulos = imp.fit_transform(datosSinPostulacion)
datosForTestFinalSinNulos = imp.fit_transform(datosToPredict)

In [246]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(datosForTrainingSinPostuSinNulos, datosML['sepostulo'], test_size=0.7, random_state=7)

In [330]:
models = []
models.append(('RFC100',  RandomForestClassifier(n_estimators=100, max_features='sqrt')))
#models.append(('RFR100',  RandomForestRegressor(n_estimators=100, max_features='sqrt')))
#models.append(('DES-TREE',tree.DecisionTreeClassifier()))


for name, model in models:
    m = model.fit(X_train, Y_train)
    pruebas = m.predict(datosForTestFinalSinNulos)
    archivo = pd.DataFrame()
    archivo['id'] = testingSet.loc[:,['id']]
    archivo['sepostulo'] = pruebas
    archivo.to_csv('entregas/archivolimpio/'+'prueba1-1-'+ name +'.csv', index=False)
    

In [326]:
from sklearn.model_selection import cross_val_score


238482

----0-----


In [ ]:
RFCModel = RandomForestClassifier(n_estimators=100, max_features='sqrt')
scores = cross_val_score(RFCModel, datosForTrainingSinPostuSinNulos, datosML['sepostulo'], cv=5)
scores

In [ ]:
RFRModel = RandomForestRegressor(n_estimators=100, max_features='sqrt')
scores = cross_val_score(RFRModel, datosForTrainingSinPostuSinNulos, datosML['sepostulo'], cv=5)
scores

----0-----


KeyError: 'idaviso'